In [ ]:
# CODE FOR 1981-1992 years of LETKF



import numpy as np
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from netCDF4 import Dataset
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import xarray as xr
import glob
from datetime import datetime, timedelta
from numba import jit

@jit()
def rms(true,prediction):
    return np.sqrt(np.nanmean((prediction-true)**2))

@jit()
def rms_tendency(variable,hours):
    variable_tendency = np.zeros((hours))
    variable = np.exp(variable) * 1000.0

    for i in range(hours):
        variable_tendency[i] = np.sqrt(np.mean((variable[i+1] - variable[i])**2.0))

    return variable_tendency

def latituded_weighted_rmse(true,prediction,lats):
    diff = prediction-true

    weights = np.cos(np.deg2rad(lats))

    weights2d = np.zeros(np.shape(diff))

    diff_squared = diff**2.0
    #weights = np.ones((10,96))

    weights2d = np.tile(weights,(96,1))
    weights2d = np.transpose(weights2d)

    masked = np.ma.MaskedArray(diff_squared, mask=np.isnan(diff_squared))
    weighted_average = np.ma.average(masked,weights=weights2d)

    return np.sqrt(weighted_average)

start_year = 1981
end_year = 1992

startdate = datetime(1981,1,1,0)
enddate = datetime(1992,1,1,0)

# create empty list to store indiviudal datasets
era5sets = []

# loop over the range of years and open each ds
for year in range(start_year, end_year + 1):
    nature_file = f'/skydata2/troyarcomano/ERA_5/{year}/era_5_y{year}_regridded_mpi_fixed_var.nc'
    ds_nature = xr.open_dataset(nature_file)
    era5sets.append(ds_nature)
ds_nature = xr.concat(era5sets, dim = 'Timestep')
ds_nature = ds_nature.sortby('Timestep')

#nature_file = f'/skydata2/troyarcomano/ERA_5/{start_year}/era_5_y{start_year}_regridded_mpi_fixed_var.nc'

analysis_file_speedy = '/skydata2/dylanelliott/letkf-hybrid-speedy/DATA/uniform_letkf_anal/uniform_analysis_81_12.nc'
analysis_file = '/skydata2/dylanelliott/letkf-hybrid-speedy/DATA/uniform_letkf_anal/hybrid_1_5_1_3_19810101_20100101/mean.nc'
spread_file =  '/skydata2/troyarcomano/letkf-hybrid-speedy/experiments/hybrid_first_test/anal_sprd.nc'

#ds_nature = xr.open_dataset(nature_file)
ds_analysis_mean = xr.open_dataset(analysis_file)
ds_analysis_mean_speedy = xr.open_dataset(analysis_file_speedy)
#ds_spread = xr.open_dataset(spread_file)
#print(ds_nature)



lats = ds_nature.Lat

level = 0.2 #0.2#0.95#0.51
level_era = 2 #2#7 #4

time_slice = slice(startdate,enddate)

var_era = 'V-wind'#'Specific_Humidity'#'Temperature' #'V-wind'
var_da =  'v'#'q'#'t'#'v'
temp_500_nature = ds_nature[var_era].sel(Sigma_Level=level_era).values
temp_500_analysis = ds_analysis_mean[var_da].sel(lev=level).values
temp_500_analysis_speedy = ds_analysis_mean_speedy[var_da].sel(lev=level,time=time_slice).values
#temp_500_spread = ds_spread[var_da].sel(lev=level).values

print('era5 shape = ',np.shape(temp_500_nature))
print('speedy shape = ',np.shape(temp_500_analysis_speedy))
print('hybrid shape = ',np.shape(temp_500_analysis))

#find smallest index value to set that as the "length"
speedy_index = temp_500_analysis_speedy.shape[0]
nature_index = temp_500_nature.shape[0]
hybrid_index = temp_500_analysis.shape[0]
smallest_index = min(speedy_index,nature_index,hybrid_index)

if smallest_index == speedy_index:
    length = speedy_index #- 1
elif smallest_index == nature_index:
    length = nature_index
else:
    length = hybrid_index
print('the smallest length is',length)

#ps_nature = ds_nature['logp'].values
#ps_nature = 1000.0 * np.exp(ps_nature)
#ps_analysis = ds_analysis_mean['ps'].values/100.0

xgrid = 96
ygrid = 48
#length =365*4*2 #1952-7 # 240 for 3 months  #1450 ##338 #160#64#177#1400#455

analysis_rmse = np.zeros((length))
analysis_rmse_speedy = np.zeros((length))
#global_average_ensemble_spread= np.zeros((length))
#ps_rmse = np.zeros((length))

analysis_error = np.zeros((length,ygrid,xgrid))
analysis_error_speedy = np.zeros((length,ygrid,xgrid))

print(np.shape(analysis_error))
print(np.shape(analysis_error_speedy))
for i in range(length):
    analysis_rmse[i] = latituded_weighted_rmse(temp_500_nature[i*6,:,:],temp_500_analysis[i,:,:],lats)
    analysis_rmse_speedy[i] = latituded_weighted_rmse(temp_500_nature[i*6,:,:],temp_500_analysis_speedy[i,:,:],lats)
    #ps_rmse[i] = rms(ps_nature[i*6,:,:],ps_analysis[i,:,:])
    analysis_error[i,:,:] = temp_500_analysis[i,:,:] - temp_500_nature[i*6,:,:]
    analysis_error_speedy[i,:,:] = temp_500_analysis_speedy[i,:,:] - temp_500_nature[i*6,:,:]
    #global_average_ensemble_spread[i] = np.average(temp_500_spread[i,:,:])

''' 24(below) instead of 28 to cut transient event (ML spin up) out in first few weeks '''
  
averaged_error = np.average(abs(analysis_error[24::,:,:]),axis=0)
averaged_error_speedy = np.average(abs(analysis_error_speedy[24::,:,:]),axis=0)

lat = ds_analysis_mean.lat.values
lon = ds_analysis_mean.lon.values

lons2d, lats2d = np.meshgrid(lon,lat)

fig = plt.figure(figsize=(6,10))
''' ax1 ===>  Makes map of hybrid letkf analysis error  '''
ax1 = plt.subplot(311,projection=ccrs.PlateCarree())
ax1.coastlines()

''' Multiply averaged_error by 1000 for spec_humid only'''

cyclic_data, cyclic_lons = add_cyclic_point(averaged_error, coord=lon)
lons2d,lats2d = np.meshgrid(cyclic_lons,lat)

cf = ax1.contourf(lons2d, lats2d,cyclic_data,levels=np.arange(0,10.1,.05),extend='both')
plt.colorbar(cf,label='(m/s)')
ax1.set_title('Hybrid 1.5,1.3 LETKF Analysis Error\n 200 hPa Meridional Wind')

diff = averaged_error - averaged_error_speedy
cyclic_data, cyclic_lons = add_cyclic_point(diff, coord=lon)
lons2d,lats2d = np.meshgrid(cyclic_lons,lat)
'''ax2 ===>  makes plot of speedy letkf analysis error '''
ax2 = plt.subplot(312,projection=ccrs.PlateCarree())
ax2.coastlines()

cyclic_data, cyclic_lons = add_cyclic_point(averaged_error_speedy, coord=lon)
lons2d,lats2d = np.meshgrid(cyclic_lons,lat)

cf = ax2.contourf(lons2d, lats2d,cyclic_data,levels=np.arange(0,10.1,.05),extend='both')
plt.colorbar(cf,label='(m/s)')
ax2.set_title('SPEEDY 1.3 LETKF Analysis Error \n 200 hPa Meridional Wind')

diff = averaged_error - averaged_error_speedy
cyclic_data, cyclic_lons = add_cyclic_point(diff, coord=lon)
lons2d,lats2d = np.meshgrid(cyclic_lons,lat)

'''ax3 ==> Makes map of difference of hybrid and speedy '''

ax3 = plt.subplot(313,projection=ccrs.PlateCarree())
ax3.coastlines()
ax3.set_title('Difference (Hybrid - SPEEDY)')
cf = ax3.contourf(lons2d, lats2d,cyclic_data,levels=np.arange(-2,2.1,.01),extend='both',cmap='seismic')

#cf = ax3.contourf(lons2d, lats2d,cyclic_data,levels=np.arange(-5,5.1,.5),extend='both',cmap='seismic')
plt.colorbar(cf,label='(m/s)')
plt.show()


#print('Average RMSE Surface Pressure (hPa)',np.average(ps_rmse))
#print('Average RMSE Variable',np.average(ps_rmse))
x = np.arange(0,length)

base = datetime(1981,1,1,0)

date_list = [base + timedelta(days=x/4) for x in range(length)]
plt.plot(date_list,analysis_rmse,color='r',label='RMSE Hybrid 1.5,1.3') #cov-infl1.3
plt.plot(date_list,analysis_rmse_speedy,color='b',label='RMSE SPEEDY 1.3')
plt.axhline(y=np.average(analysis_rmse[20::]), color='r', linestyle='--',linewidth=.1,label="Average RMSE Hybrid 1.5,1.3")
plt.axhline(y=np.average(analysis_rmse_speedy[20::]), color='b', linestyle='--',linewidth=.1,label="Average RMSE SPEEDY 1.3")
print('average rmse Hybrid', np.average(analysis_rmse[20::]))
print('average rmse SPEEDY', np.average(analysis_rmse_speedy[20::]))
#plt.plot(date_list,global_average_ensemble_spread,label='Ensemble Spread')
#plt.title('LETKF Analysis Error\n Low Level Specific Humidity')
plt.title('LETKF Analysis Error\n 200 hPa Meridional Wind')
#plt.title('Ensemble Spread\nModel Level 4 Temperature')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Analysis Error')
#plt.ylabel('Analysis Error (kg/kg)')
#plt.ylabel('Ensemble Spread (K)')
plt.xlim([datetime(1981, 1, 1,0), datetime(1993, 2, 1,0)])
plt.ylim(3,10)
plt.show()
    

## plot of spin-ups
#x= np.arange(0,length)
#
#date_list = [base + timedelta(days=x/4) for x in range(length)]
#plt.plot(date_list,analysis_rmse,color='r',label='RMSE Hybrid ') #cov-infl1.3
#plt.plot(date_list,analysis_rmse_speedy,color='b',label='RMSE SPEEDY')
#plt.axhline(y=np.average(analysis_rmse[20::]), color='r', linestyle='--',label="Average RMSE Hybrid")
#plt.axhline(y=np.average(analysis_rmse_speedy[20::]), color='b', linestyle='--',label="Average RMSE SPEEDY")
#print('average rmse Hybrid', np.average(analysis_rmse[20::])) #20
#print('average rmse SPEEDY', np.average(analysis_rmse_speedy[20::])) #20
##plt.plot(date_list,global_average_ensemble_spread,label='Ensemble Spread')
##plt.title('LETKF Analysis Error\n Low Level Specific Humidity')
#plt.title('LETKF Analysis Error\n 200 hPa Meridional Wind\n Initial Spin-up')
##plt.title('Ensemble Spread\nModel Level 4 Temperature')
#plt.legend()
#plt.xlabel('Date')
#plt.ylabel('Analysis Error')
##plt.ylabel('Analysis Error (kg/kg)')
##plt.ylabel('Ensemble Spread (K)')
#plt.xlim([datetime(2012, 1, 1,0), datetime(2012, 1, 4,0)])
#plt.show()
# 


In [ ]:
x = np.arange(0,length)

base = datetime(1981,1,1,0)
plt.figure(figsize=(12,6))
date_list = [base + timedelta(days=x/4) for x in range(length)]
plt.plot(date_list,analysis_rmse,color='r',linewidth=.15,label='RMSE Hybrid 1.5,1.3') #cov-infl1.3
plt.plot(date_list,analysis_rmse_speedy,color='b',linewidth=.15,label='RMSE SPEEDY 1.3')
plt.axhline(y=np.average(analysis_rmse[20::]), color='r', linestyle='--',linewidth=1,label="Average RMSE Hybrid 1.5,1.3")
plt.axhline(y=np.average(analysis_rmse_speedy[20::]), color='b', linestyle='--',linewidth=1,label="Average RMSE SPEEDY 1.3")
print('average rmse Hybrid = ', np.average(analysis_rmse[20::]))
print('average rmse SPEEDY = ', np.average(analysis_rmse_speedy[20::]))
#plt.plot(date_list,global_average_ensemble_spread,label='Ensemble Spread')
#plt.title('LETKF Analysis Error\n Low Level Specific Humidity')
plt.title('LETKF Analysis Error\n 200 hPa Meridional Wind')
#plt.title('Ensemble Spread\nModel Level 4 Temperature')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Analysis Error')
#plt.ylabel('Analysis Error (kg/kg)')
#plt.ylabel('Ensemble Spread (K)')
plt.xlim([datetime(1981, 1, 1,0), datetime(1992, 2, 1,0)])
plt.ylim(3,7)
plt.show()

In [1]:
# CODE FOR 1992-2003 LETKF 

import numpy as np
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from netCDF4 import Dataset
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import xarray as xr
import glob
from datetime import datetime, timedelta
from numba import jit

@jit()
def rms(true,prediction):
    return np.sqrt(np.nanmean((prediction-true)**2))

@jit()
def rms_tendency(variable,hours):
    variable_tendency = np.zeros((hours))
    variable = np.exp(variable) * 1000.0

    for i in range(hours):
        variable_tendency[i] = np.sqrt(np.mean((variable[i+1] - variable[i])**2.0))

    return variable_tendency

def latituded_weighted_rmse(true,prediction,lats):
    diff = prediction-true

    weights = np.cos(np.deg2rad(lats))

    weights2d = np.zeros(np.shape(diff))

    diff_squared = diff**2.0
    #weights = np.ones((10,96))

    weights2d = np.tile(weights,(96,1))
    weights2d = np.transpose(weights2d)

    masked = np.ma.MaskedArray(diff_squared, mask=np.isnan(diff_squared))
    weighted_average = np.ma.average(masked,weights=weights2d)

    return np.sqrt(weighted_average)

start_year = 1981
end_year = 1993

startdate = datetime(1992,1,1,0)
enddate = datetime(2003,1,1,0)

# create empty list to store indiviudal datasets
era5sets = []

# loop over the range of years and open each ds
for year in range(start_year, end_year + 1):
    nature_file = f'/skydata2/troyarcomano/ERA_5/{year}/era_5_y{year}_regridded_mpi_fixed_var.nc'
    ds_nature = xr.open_dataset(nature_file)
    era5sets.append(ds_nature)
ds_nature = xr.concat(era5sets, dim = 'Timestep')
ds_nature = ds_nature.sortby('Timestep')

#nature_file = f'/skydata2/troyarcomano/ERA_5/{start_year}/era_5_y{start_year}_regridded_mpi_fixed_var.nc'

analysis_file_speedy = '/skydata2/dylanelliott/letkf-hybrid-speedy/DATA/uniform_letkf_anal/uniform_analysis_81_12.nc'
analysis_file = '/skydata2/dylanelliott/letkf-hybrid-speedy/DATA/uniform_letkf_anal/hybrid_1_5_1_3_19810101_20100101/mean.nc'
spread_file =  '/skydata2/troyarcomano/letkf-hybrid-speedy/experiments/hybrid_first_test/anal_sprd.nc'

#ds_nature = xr.open_dataset(nature_file)
ds_analysis_mean = xr.open_dataset(analysis_file)
ds_analysis_mean_speedy = xr.open_dataset(analysis_file_speedy)
#ds_spread = xr.open_dataset(spread_file)
#print(ds_nature)



lats = ds_nature.Lat

level = 0.2 #0.2#0.95#0.51
level_era = 2 #2#7 #4

time_slice = slice(startdate,enddate)

var_era = 'V-wind'#'Specific_Humidity'#'Temperature' #'V-wind'
var_da =  'v'#'q'#'t'#'v'
temp_500_nature = ds_nature[var_era].sel(Sigma_Level=level_era).values
temp_500_analysis = ds_analysis_mean[var_da].sel(lev=level).values
temp_500_analysis_speedy = ds_analysis_mean_speedy[var_da].sel(lev=level,time=time_slice).values
#temp_500_spread = ds_spread[var_da].sel(lev=level).values

print('era5 shape = ',np.shape(temp_500_nature))
print('speedy shape = ',np.shape(temp_500_analysis_speedy))
print('hybrid shape = ',np.shape(temp_500_analysis))

#find smallest index value to set that as the "length"
speedy_index = temp_500_analysis_speedy.shape[0]
nature_index = temp_500_nature.shape[0]
hybrid_index = temp_500_analysis.shape[0]
smallest_index = min(speedy_index,nature_index,hybrid_index)

if smallest_index == speedy_index:
    length = speedy_index #- 1
elif smallest_index == nature_index:
    length = nature_index
else:
    length = hybrid_index
print('the smallest length is',length)

#ps_nature = ds_nature['logp'].values
#ps_nature = 1000.0 * np.exp(ps_nature)
#ps_analysis = ds_analysis_mean['ps'].values/100.0

xgrid = 96
ygrid = 48
#length =365*4*2 #1952-7 # 240 for 3 months  #1450 ##338 #160#64#177#1400#455

analysis_rmse = np.zeros((length))
analysis_rmse_speedy = np.zeros((length))
#global_average_ensemble_spread= np.zeros((length))
#ps_rmse = np.zeros((length))

analysis_error = np.zeros((length,ygrid,xgrid))
analysis_error_speedy = np.zeros((length,ygrid,xgrid))

print(np.shape(analysis_error))
print(np.shape(analysis_error_speedy))
for i in range(length):
    analysis_rmse[i] = latituded_weighted_rmse(temp_500_nature[i*6,:,:],temp_500_analysis[i,:,:],lats)
    analysis_rmse_speedy[i] = latituded_weighted_rmse(temp_500_nature[i*6,:,:],temp_500_analysis_speedy[i,:,:],lats)
    #ps_rmse[i] = rms(ps_nature[i*6,:,:],ps_analysis[i,:,:])
    analysis_error[i,:,:] = temp_500_analysis[i,:,:] - temp_500_nature[i*6,:,:]
    analysis_error_speedy[i,:,:] = temp_500_analysis_speedy[i,:,:] - temp_500_nature[i*6,:,:]
    #global_average_ensemble_spread[i] = np.average(temp_500_spread[i,:,:])

''' 24(below) instead of 28 to cut transient event (ML spin up) out in first few weeks '''
  
averaged_error = np.average(abs(analysis_error[24::,:,:]),axis=0)
averaged_error_speedy = np.average(abs(analysis_error_speedy[24::,:,:]),axis=0)

lat = ds_analysis_mean.lat.values
lon = ds_analysis_mean.lon.values

lons2d, lats2d = np.meshgrid(lon,lat)

fig = plt.figure(figsize=(6,10))
''' ax1 ===>  Makes map of hybrid letkf analysis error  '''
ax1 = plt.subplot(311,projection=ccrs.PlateCarree())
ax1.coastlines()

''' Multiply averaged_error by 1000 for spec_humid only'''

cyclic_data, cyclic_lons = add_cyclic_point(averaged_error, coord=lon)
lons2d,lats2d = np.meshgrid(cyclic_lons,lat)

cf = ax1.contourf(lons2d, lats2d,cyclic_data,levels=np.arange(0,10.1,.05),extend='both')
plt.colorbar(cf,label='(m/s)')
ax1.set_title('Hybrid 1.5,1.3 LETKF Analysis Error\n 200 hPa Meridional Wind')

diff = averaged_error - averaged_error_speedy
cyclic_data, cyclic_lons = add_cyclic_point(diff, coord=lon)
lons2d,lats2d = np.meshgrid(cyclic_lons,lat)
'''ax2 ===>  makes plot of speedy letkf analysis error '''
ax2 = plt.subplot(312,projection=ccrs.PlateCarree())
ax2.coastlines()

cyclic_data, cyclic_lons = add_cyclic_point(averaged_error_speedy, coord=lon)
lons2d,lats2d = np.meshgrid(cyclic_lons,lat)

cf = ax2.contourf(lons2d, lats2d,cyclic_data,levels=np.arange(0,10.1,.05),extend='both')
plt.colorbar(cf,label='(m/s)')
ax2.set_title('SPEEDY 1.3 LETKF Analysis Error \n 200 hPa Meridional Wind')

diff = averaged_error - averaged_error_speedy
cyclic_data, cyclic_lons = add_cyclic_point(diff, coord=lon)
lons2d,lats2d = np.meshgrid(cyclic_lons,lat)

'''ax3 ==> Makes map of difference of hybrid and speedy '''

ax3 = plt.subplot(313,projection=ccrs.PlateCarree())
ax3.coastlines()
ax3.set_title('Difference (Hybrid - SPEEDY)')
cf = ax3.contourf(lons2d, lats2d,cyclic_data,levels=np.arange(-2,2.1,.01),extend='both',cmap='seismic')

#cf = ax3.contourf(lons2d, lats2d,cyclic_data,levels=np.arange(-5,5.1,.5),extend='both',cmap='seismic')
plt.colorbar(cf,label='(m/s)')
plt.show()


#print('Average RMSE Surface Pressure (hPa)',np.average(ps_rmse))
#print('Average RMSE Variable',np.average(ps_rmse))
x = np.arange(0,length)

base = datetime(1992,1,1,0)
plt.figure(figsize=(12,6))
date_list = [base + timedelta(days=x/4) for x in range(length)]
plt.plot(date_list,analysis_rmse,color='r',linewidth=.15,label='RMSE Hybrid 1.5,1.3') #cov-infl1.3
plt.plot(date_list,analysis_rmse_speedy,color='b',linewidth=.15,label='RMSE SPEEDY 1.3')
plt.axhline(y=np.average(analysis_rmse[20::]), color='r', linestyle='--',linewidth=1,label="Average RMSE Hybrid 1.5,1.3")
plt.axhline(y=np.average(analysis_rmse_speedy[20::]), color='b', linestyle='--',linewidth=1,label="Average RMSE SPEEDY 1.3")
print('average rmse Hybrid = ', np.average(analysis_rmse[20::]))
print('average rmse SPEEDY = ', np.average(analysis_rmse_speedy[20::]))
#plt.plot(date_list,global_average_ensemble_spread,label='Ensemble Spread')
#plt.title('LETKF Analysis Error\n Low Level Specific Humidity')
plt.title('LETKF Analysis Error\n 200 hPa Meridional Wind')
#plt.title('Ensemble Spread\nModel Level 4 Temperature')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Analysis Error')
#plt.ylabel('Analysis Error (kg/kg)')
#plt.ylabel('Ensemble Spread (K)')
plt.xlim([datetime(1992, 1, 1,0), datetime(2003, 2, 1,0)])
plt.ylim(3,7)
plt.show()
    

## plot of spin-ups
#x= np.arange(0,length)
#
#date_list = [base + timedelta(days=x/4) for x in range(length)]
#plt.plot(date_list,analysis_rmse,color='r',label='RMSE Hybrid ') #cov-infl1.3
#plt.plot(date_list,analysis_rmse_speedy,color='b',label='RMSE SPEEDY')
#plt.axhline(y=np.average(analysis_rmse[20::]), color='r', linestyle='--',label="Average RMSE Hybrid")
#plt.axhline(y=np.average(analysis_rmse_speedy[20::]), color='b', linestyle='--',label="Average RMSE SPEEDY")
#print('average rmse Hybrid', np.average(analysis_rmse[20::])) #20
#print('average rmse SPEEDY', np.average(analysis_rmse_speedy[20::])) #20
##plt.plot(date_list,global_average_ensemble_spread,label='Ensemble Spread')
##plt.title('LETKF Analysis Error\n Low Level Specific Humidity')
#plt.title('LETKF Analysis Error\n 200 hPa Meridional Wind\n Initial Spin-up')
##plt.title('Ensemble Spread\nModel Level 4 Temperature')
#plt.legend()
#plt.xlabel('Date')
#plt.ylabel('Analysis Error')
##plt.ylabel('Analysis Error (kg/kg)')
##plt.ylabel('Ensemble Spread (K)')
#plt.xlim([datetime(2012, 1, 1,0), datetime(2012, 1, 4,0)])
#plt.show()
# 

KeyboardInterrupt: 

In [ ]:
x = np.arange(0,length)

base = datetime(1992,1,1,0)
plt.figure(figsize=(12,6))
date_list = [base + timedelta(days=x/4) for x in range(length)]
plt.plot(date_list,analysis_rmse,color='r',linewidth=.15,label='RMSE Hybrid 1.5,1.3') #cov-infl1.3
plt.plot(date_list,analysis_rmse_speedy,color='b',linewidth=.15,label='RMSE SPEEDY 1.3')
plt.axhline(y=np.average(analysis_rmse[20::]), color='r', linestyle='--',linewidth=1,label="Average RMSE Hybrid 1.5,1.3")
plt.axhline(y=np.average(analysis_rmse_speedy[20::]), color='b', linestyle='--',linewidth=1,label="Average RMSE SPEEDY 1.3")
print('average rmse Hybrid = ', np.average(analysis_rmse[20::]))
print('average rmse SPEEDY = ', np.average(analysis_rmse_speedy[20::]))
#plt.plot(date_list,global_average_ensemble_spread,label='Ensemble Spread')
#plt.title('LETKF Analysis Error\n Low Level Specific Humidity')
plt.title('LETKF Analysis Error\n 200 hPa Meridional Wind')
#plt.title('Ensemble Spread\nModel Level 4 Temperature')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Analysis Error')
#plt.ylabel('Analysis Error (kg/kg)')
#plt.ylabel('Ensemble Spread (K)')
plt.xlim([datetime(1992, 1, 1,0), datetime(2003, 2, 1,0)])
plt.ylim(3,7)
plt.show()